In [95]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/merged-caltech-weather-traffic/merged_data_wt2.csv
/kaggle/input/merged-caltech-weather-traffic/merged_data_wt.csv
/kaggle/input/caltech1/caltech1.csv


In [96]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [97]:
df = pd.read_csv('/kaggle/input/merged-caltech-weather-traffic/merged_data_wt2.csv')


In [98]:
df.columns

Index(['Unnamed: 0', 'sessionID', 'stationID', 'spaceID', 'connectionTime',
       'disconnectTime', 'kWhDelivered', 'doneChargingTime', 'timezone',
       'userID', 'session_duration', 'Weekend', 'connectionDate', 'clouds',
       'datetime', 'dewpt', 'max_uv', 'max_wind_spd', 'min_temp', 'precip',
       'pres', 'revision_status', 'rh', 'slp', 'temp', 'wind_dir', 'wind_spd',
       'Date', 'TrafficVolume', 'AverageSpeed'],
      dtype='object')

In [99]:
df = df.drop(['Unnamed: 0','sessionID','connectionTime', 'disconnectTime','doneChargingTime','timezone','datetime','connectionDate', 'Date', 'revision_status','stationID'], axis=1)

In [100]:
df.shape

(1945, 19)

In [101]:
df.columns

Index(['spaceID', 'kWhDelivered', 'userID', 'session_duration', 'Weekend',
       'clouds', 'dewpt', 'max_uv', 'max_wind_spd', 'min_temp', 'precip',
       'pres', 'rh', 'slp', 'temp', 'wind_dir', 'wind_spd', 'TrafficVolume',
       'AverageSpeed'],
      dtype='object')

In [102]:
df.head()

,spaceID,kWhDelivered,userID,session_duration,Weekend,clouds,dewpt,max_uv,max_wind_spd,min_temp,precip,pres,rh,slp,temp,wind_dir,wind_spd,TrafficVolume,AverageSpeed
0,CA-305,39.894,claimed,1049.750000,0,51,1.6,2.9,3.4,1.5,0.8,987,80.0,1011,5.0,172,1.4,175.16875,34.242583
1,CA-314,10.627,claimed,198.066667,0,51,1.6,2.9,3.4,1.5,0.8,987,80.0,1011,5.0,172,1.4,175.16875,34.242583
2,CA-303,30.479,claimed,288.916667,1,92,1.6,1.5,2.4,2.4,1.5,997,73.5,1021,6.3,53,1.6,175.16875,34.242583
3,CA-315,1.650,claimed,92.283333,1,92,1.6,1.5,2.4,2.4,1.5,997,73.5,1021,6.3,53,1.6,175.16875,34.242583
4,CA-313,7.049,claimed,127.866667,1,92,1.6,1.5,2.4,2.4,1.5,997,73.5,1021,6.3,53,1.6,175.16875,34.242583


In [103]:
df.columns

Index(['spaceID', 'kWhDelivered', 'userID', 'session_duration', 'Weekend',
       'clouds', 'dewpt', 'max_uv', 'max_wind_spd', 'min_temp', 'precip',
       'pres', 'rh', 'slp', 'temp', 'wind_dir', 'wind_spd', 'TrafficVolume',
       'AverageSpeed'],
      dtype='object')

In [104]:
data = df

In [105]:
# Identify categorical columns based on data types
data_types = data.dtypes
categorical_columns = data_types[data_types == 'object'].index.tolist()

# Check unique values in categorical columns to confirm if they are indeed categorical
for column in categorical_columns:
    unique_values = data[column].unique()
    print(f"Unique values in {column}: {unique_values}")

# You can also set a threshold to determine if a column is categorical based on the number of unique values
threshold_unique_values = 10  # Adjust threshold as needed
categorical_columns = [column for column in data.columns if len(data[column].unique()) <= threshold_unique_values]

# Print the identified categorical columns
print("Categorical Columns:", categorical_columns)

# Non-categorical columns
non_categorical_columns = [column for column in data.columns if column not in categorical_columns]
print("Non-Categorical Columns:", non_categorical_columns)

Unique values in spaceID: ['CA-305' 'CA-314' 'CA-303' 'CA-315' 'CA-313' 'CA-307' 'CA-317' 'CA-309'
 'CA-325' 'CA-491' 'CA-323' 'CA-506' 'CA-319' 'CA-327' 'CA-326' 'CA-322'
 'CA-498' 'CA-490' 'CA-493' 'CA-496' 'CA-499' 'CA-320' 'CA-497' 'CA-316'
 'CA-312' 'CA-502' 'CA-495' 'CA-489' 'CA-304' 'CA-494' 'CA-306' 'CA-511'
 'CA-308' 'CA-321' 'CA-492' 'CA-318' 'CA-324' 'CA-500' 'CA-310' 'CA-311'
 'CA-507' 'CA-503' 'CA-504' 'CA-508' 'CA-501' 'CA-509' 'CA-212' 'CA-510'
 'CA-213' 'CA-512']
Unique values in userID: ['claimed' 'unclaimed']
Categorical Columns: ['userID', 'Weekend']
Non-Categorical Columns: ['spaceID', 'kWhDelivered', 'session_duration', 'clouds', 'dewpt', 'max_uv', 'max_wind_spd', 'min_temp', 'precip', 'pres', 'rh', 'slp', 'temp', 'wind_dir', 'wind_spd', 'TrafficVolume', 'AverageSpeed']


In [106]:
import pandas as pd

# Assuming 'data' is your DataFrame
# List of categorical columns
categorical_columns = ['spaceID', 'userID']

# Apply one-hot encoding for each categorical column
for column in categorical_columns:
    # Convert the column to one-hot encoded representation
    one_hot_encoded = pd.get_dummies(data[column], prefix=column)
    # Concatenate the one-hot encoded columns to the original dataframe
    data = pd.concat([data, one_hot_encoded], axis=1)
    # Drop the original categorical column from the dataframe
    data.drop(column, axis=1, inplace=True)


In [107]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import IsolationForest
from sklearn.decomposition import PCA
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor

# Load the data


# Data Cleaning and Preprocessing

# Drop any rows with missing values
#data.dropna(inplace=True)

# Convert categorical variables to dummy/indicator variables if needed


# Null Value Imputation    
imputer = SimpleImputer(strategy='mean')
data_imputed = imputer.fit_transform(data)
data_imputed = pd.DataFrame(data_imputed, columns=data.columns)


# # Outlier Detection and Removal
# outlier_detector = IsolationForest(contamination=0.5)  # Adjust contamination as needed
# outliers = outlier_detector.fit_predict(data_imputed)
# data_cleaned = data_imputed[outliers == 1]
data_imputed = data_imputed.drop(['session_duration'], axis=1)
# # Count the number of outliers
# num_outliers = len(outliers[outliers == -1])
# print("Number of outliers detected and removed:", num_outliers)
# # Dimension Reduction
scaler = StandardScaler()
data_scaled = scaler.fit_transform(data_imputed)


pca = PCA(n_components=15)  
data_pca = pca.fit_transform(data_scaled)



In [108]:
data.columns

Index(['kWhDelivered', 'session_duration', 'Weekend', 'clouds', 'dewpt',
       'max_uv', 'max_wind_spd', 'min_temp', 'precip', 'pres', 'rh', 'slp',
       'temp', 'wind_dir', 'wind_spd', 'TrafficVolume', 'AverageSpeed',
       'spaceID_CA-212', 'spaceID_CA-213', 'spaceID_CA-303', 'spaceID_CA-304',
       'spaceID_CA-305', 'spaceID_CA-306', 'spaceID_CA-307', 'spaceID_CA-308',
       'spaceID_CA-309', 'spaceID_CA-310', 'spaceID_CA-311', 'spaceID_CA-312',
       'spaceID_CA-313', 'spaceID_CA-314', 'spaceID_CA-315', 'spaceID_CA-316',
       'spaceID_CA-317', 'spaceID_CA-318', 'spaceID_CA-319', 'spaceID_CA-320',
       'spaceID_CA-321', 'spaceID_CA-322', 'spaceID_CA-323', 'spaceID_CA-324',
       'spaceID_CA-325', 'spaceID_CA-326', 'spaceID_CA-327', 'spaceID_CA-489',
       'spaceID_CA-490', 'spaceID_CA-491', 'spaceID_CA-492', 'spaceID_CA-493',
       'spaceID_CA-494', 'spaceID_CA-495', 'spaceID_CA-496', 'spaceID_CA-497',
       'spaceID_CA-498', 'spaceID_CA-499', 'spaceID_CA-500', 'spaceI

In [109]:
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1945 entries, 0 to 1944
Data columns (total 69 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   kWhDelivered      1945 non-null   float64
 1   session_duration  1945 non-null   float64
 2   Weekend           1945 non-null   int64  
 3   clouds            1945 non-null   int64  
 4   dewpt             1945 non-null   float64
 5   max_uv            1945 non-null   float64
 6   max_wind_spd      1945 non-null   float64
 7   min_temp          1945 non-null   float64
 8   precip            1945 non-null   float64
 9   pres              1945 non-null   int64  
 10  rh                1945 non-null   float64
 11  slp               1945 non-null   int64  
 12  temp              1945 non-null   float64
 13  wind_dir          1945 non-null   int64  
 14  wind_spd          1945 non-null   float64
 15  TrafficVolume     1945 non-null   float64
 16  AverageSpeed      1945 non-null   float64


In [115]:
X = data.drop(columns=['session_duration'])
#X = data_pca
y = data['session_duration']

In [111]:
# numeric_columns = X.select_dtypes(include=['number']).columns.tolist()
# object_columns = X.select_dtypes(include=['object']).columns.tolist()

# print("Numeric Columns:", numeric_columns)
# print("Object Columns:", object_columns)


In [116]:
X

,kWhDelivered,Weekend,clouds,dewpt,max_uv,max_wind_spd,min_temp,precip,pres,rh,...,spaceID_CA-504,spaceID_CA-506,spaceID_CA-507,spaceID_CA-508,spaceID_CA-509,spaceID_CA-510,spaceID_CA-511,spaceID_CA-512,userID_claimed,userID_unclaimed
0,39.894,0,51,1.6,2.9,3.4,1.5,0.8,987,80.0,...,False,False,False,False,False,False,False,False,True,False
1,10.627,0,51,1.6,2.9,3.4,1.5,0.8,987,80.0,...,False,False,False,False,False,False,False,False,True,False
2,30.479,1,92,1.6,1.5,2.4,2.4,1.5,997,73.5,...,False,False,False,False,False,False,False,False,True,False
3,1.650,1,92,1.6,1.5,2.4,2.4,1.5,997,73.5,...,False,False,False,False,False,False,False,False,True,False
4,7.049,1,92,1.6,1.5,2.4,2.4,1.5,997,73.5,...,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1940,6.298,0,4,11.6,11.1,3.2,16.1,0.0,992,52.0,...,False,False,False,False,False,False,False,False,True,False
1941,4.034,0,4,11.6,11.1,3.2,16.1,0.0,992,52.0,...,False,False,False,False,False,False,False,False,True,False
1942,0.812,0,4,11.6,11.1,3.2,16.1,0.0,992,52.0,...,False,False,False,False,False,False,False,False,True,False
1943,6.423,0,0,3.9,10.8,3.6,23.2,0.0,990,20.6,...,False,False,False,False,False,False,False,False,True,False


In [117]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from lightgbm import LGBMRegressor
from sklearn.svm import SVR
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=True)

# Machine Learning Model - Random Forest
rf_model = RandomForestRegressor(n_estimators=1000, max_depth=10)
rf_model.fit(X_train, y_train)
rf_score = rf_model.score(X_test, y_test)

# Machine Learning Model - XGBoost
xgb_model = XGBRegressor(n_estimators=1000, max_depth=10, learning_rate=0.1)
xgb_model.fit(X_train, y_train)
xgb_score = xgb_model.score(X_test, y_test)

# Machine Learning Model - LightGBM
lgbm_model = LGBMRegressor()
lgbm_model.fit(X_train, y_train)
lgbm_score = lgbm_model.score(X_test, y_test)

# Machine Learning Model - SVM
svm_model = SVR()
svm_model.fit(X_train, y_train)
svm_score = svm_model.score(X_test, y_test)

# Machine Learning Model - CatBoost
catboost_model = CatBoostRegressor(silent=True)
catboost_model.fit(X_train, y_train)
catboost_score = catboost_model.score(X_test, y_test)

# Predictions
rf_predictions = rf_model.predict(X_test)
xgb_predictions = xgb_model.predict(X_test)
lgbm_predictions = lgbm_model.predict(X_test)
svm_predictions = svm_model.predict(X_test)
catboost_predictions = catboost_model.predict(X_test)

# Calculate RMSE and MAE for Random Forest
rf_rmse = np.sqrt(mean_squared_error(y_test, rf_predictions))
rf_mae = mean_absolute_error(y_test, rf_predictions)

# Calculate RMSE and MAE for XGBoost
xgb_rmse = np.sqrt(mean_squared_error(y_test, xgb_predictions))
xgb_mae = mean_absolute_error(y_test, xgb_predictions)

# Calculate RMSE and MAE for LightGBM
lgbm_rmse = np.sqrt(mean_squared_error(y_test, lgbm_predictions))
lgbm_mae = mean_absolute_error(y_test, lgbm_predictions)

# Calculate RMSE and MAE for SVM
svm_rmse = np.sqrt(mean_squared_error(y_test, svm_predictions))
svm_mae = mean_absolute_error(y_test, svm_predictions)

# Calculate RMSE and MAE for CatBoost
catboost_rmse = np.sqrt(mean_squared_error(y_test, catboost_predictions))
catboost_mae = mean_absolute_error(y_test, catboost_predictions)

# Print the evaluation metrics
print("Random Forest Metrics:")
print("RMSE:", rf_rmse)
print("MAE:", rf_mae)
print("\nXGBoost Metrics:")
print("RMSE:", xgb_rmse)
print("MAE:", xgb_mae)
print("\nLightGBM Metrics:")
print("RMSE:", lgbm_rmse)
print("MAE:", lgbm_mae)
print("\nSVM Metrics:")
print("RMSE:", svm_rmse)
print("MAE:", svm_mae)
print("\nCatBoost Metrics:")
print("RMSE:", catboost_rmse)
print("MAE:", catboost_mae)


[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000695 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1119
[LightGBM] [Info] Number of data points in the train set: 1556, number of used features: 55
[LightGBM] [Info] Start training from score 338.274143
Random Forest Metrics:
RMSE: 211.37956814818764
MAE: 163.16467392101012

XGBoost Metrics:
RMSE: 244.04340606578583
MAE: 165.34734144874793

LightGBM Metrics:
RMSE: 218.64022325868595
MAE: 167.6706443245574

SVM Metrics:
RMSE: 229.31210055576133
MAE: 183.87633933786782

CatBoost Metrics:
RMSE: 220.2021932817096
MAE: 162.78054220892733


In [114]:
8ll

SyntaxError: invalid decimal literal (1199300867.py, line 1)

In [ ]:
# Feature Selection
model = RandomForestRegressor()
model.fit(data_pca, data_imputed['session_duration'])  # Replace 'target_column' with your target variable
feature_importances = model.feature_importances_
selector = SelectFromModel(model, prefit=True)
selected_features = selector.transform(data_imputeda)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(selected_features, data_imputed['session_duration'], test_size=0.2)

# Machine Learning Model - Random Forest
rf_model = RandomForestRegressor()
rf_model.fit(X_train, y_train)
rf_score = rf_model.score(X_test, y_test)

# Machine Learning Model - XGBoost
xgb_model = XGBRegressor()
xgb_model.fit(X_train, y_train)
xgb_score = xgb_model.score(X_test, y_test)

# print("Random Forest Score:", rf_score)
# print("XGBoost Score:", xgb_score)
from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

# Predictions
rf_predictions = rf_model.predict(X_test)
xgb_predictions = xgb_model.predict(X_test)

# Calculate RMSE and MAE for Random Forest
rf_rmse = np.sqrt(mean_squared_error(y_test, rf_predictions))
rf_mae = mean_absolute_error(y_test, rf_predictions)

# Calculate RMSE and MAE for XGBoost
xgb_rmse = np.sqrt(mean_squared_error(y_test, xgb_predictions))
xgb_mae = mean_absolute_error(y_test, xgb_predictions)

# Print the evaluation metrics
print("Random Forest Metrics:")
print("RMSE:", rf_rmse)
print("MAE:", rf_mae)
print("\nXGBoost Metrics:")
print("RMSE:", xgb_rmse)
print("MAE:", xgb_mae)



In [ ]:
selected_features